# Comparing Llama 3.1 vs. GPT-4o mini

In this notebook you will use the Meta's Llama 3.1 and OpenAI's GPT-4o mini to solve:

- Task 1: Zero-shot Classification
- Task 2: Few-shot Classification
- Task 3: Coding Tasks - Python
- Task 4: Coding Tasks - SQL
- Task 5: Information Extraction
- Task 6: Closed-Domain Question Answering
- Task 7: Open-Domain Question Answering
- Task 8: Document Summarization
- Task 9: Transformation
- Task 10: Translation



___Created By: Dipanjan (DJ)___

In [1]:
import torch

torch.cuda.empty_cache()

## Load OpenAI API Credentials


In [2]:
from getpass import getpass

openai_key = getpass("Enter your OpenAI API Key: ")

Enter your OpenAI API Key:  ········


In [3]:
import openai
from IPython.display import HTML, Markdown, display

openai.api_key = openai_key

## Load HuggingFace API Credentials


In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Create ChatGPT Completion Access Function

This function will use the [Chat Completion API](https://platform.openai.com/docs/api-reference/chat/create) to access ChatGPT for us and return responses


In [5]:
def get_completion_gpt(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0, # degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Create Llama 3.1 Completion Access Function

This function will use the transformers [pipeline](https://huggingface.co/docs/transformers/en/main_classes/pipelines) module to download and load LLama 3.1 8B for us and return responses


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id, 
                                          cache_dir='./llama3')
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
    cache_dir='./llama3',
)

llama3 = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="cuda",
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [10]:
def get_completion_llama(prompt, model_pipeline=llama3):
    messages = [{"role": "user", "content": prompt}]
    response = model_pipeline(
        messages,
        max_new_tokens=2000
    )
    return response[0]["generated_text"][-1]['content']

## Let's try out the ChatGPT API!

In [11]:
response = get_completion_gpt(prompt='Explain Generative AI in 2 bullet points')
display(Markdown(response))

- **Definition and Functionality**: Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or videos, by learning patterns and structures from existing data. These models, like GPT and DALL-E, utilize techniques such as deep learning and neural networks to generate outputs that mimic human creativity.

- **Applications and Impact**: Generative AI has a wide range of applications, including content creation, design, gaming, and personalized experiences. It can enhance productivity, enable new forms of artistic expression, and facilitate innovation across various industries, but it also raises ethical concerns regarding copyright, misinformation, and the authenticity of generated content.

## Let's try out LLama 3.1

In [12]:
response = get_completion_llama(prompt='Explain Generative AI in 2 bullet points')
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here are 2 bullet points explaining Generative AI:

• **Definition**: Generative AI refers to a type of artificial intelligence that can create new, original content such as images, music, videos, text, or even entire worlds, based on patterns and structures learned from existing data. This is achieved through complex algorithms and machine learning models that can generate novel outputs that are often indistinguishable from human-created content.

• **Applications**: Generative AI has a wide range of applications across various industries, including art, music, film, gaming, and advertising. It can be used to generate realistic images and videos, create new music and audio tracks, write stories and articles, and even design new products and experiences. Additionally, generative AI can also be used for tasks such as data augmentation, anomaly detection, and personalized content creation.

## Task 1: Zero-shot Classification

This prompt tests an LLM's text classification capabilities by prompting it to classify a piece of text without providing any examples.



In [13]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """,
    f"""
    I tried to like this book and while the plot was really good, the print quality was so not good
    """
]

In [14]:
responses = {
    'llama3.1' : [],
    'gpt-4o-mini' : []
}

for review in reviews:
  prompt = f"""
              Act as a product review analyst.
              Given the following review,
              Display the overall sentiment for the review as only one of the following:
              Positive, Negative OR Neutral

              Just give me the sentiment only.

              ```{review}```
              """
  response = get_completion_llama(prompt)
  responses['llama3.1'].append(response)
  response = get_completion_gpt(prompt)
  responses['gpt-4o-mini'].append(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.DataFrame(responses)

,llama3.1,gpt-4o-mini
0,Positive,Positive
1,Negative,Negative
2,Negative,Negative


## Task 2: Few-shot Classification

This prompt tests an LLM's text classification capabilities by prompting it to classify a piece of text by providing a few examples of inputs and outputs.



In [16]:
responses = {
    'llama3.1' : [],
    'gpt-4o-mini' : []
}


for review in reviews:
  prompt = f"""
              Act as a product review analyst.
              Given the following review,
              Display only the sentiment for the review:

              Try to classify it by using the following examples as a reference:

              Review: Just received the Laptop I ordered for work, and it's amazing.
              Sentiment: 😊

              Review: Needed a new mechanical keyboard, but this model has been totally disappointing.
              Sentiment: 😡

              Review: ```{review}```
              Sentiment:
              """
  response = get_completion_llama(prompt)
  responses['llama3.1'].append(response)
  response = get_completion_gpt(prompt)
  responses['gpt-4o-mini'].append(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [17]:
pd.DataFrame(responses)

,llama3.1,gpt-4o-mini
0,"Based on the reference examples, I would classify the sentiment of the review as:\n\n😊",😊
1,"Based on the given examples and the review, I would classify the sentiment as:\n\n😡\n\nThe review expresses several negative points about the product, including its performance, noise level, and design flaws. The reviewer also mentions that the product did not meet their expectations and was not worth the expense. These negative comments and the use of phrases like ""nightmare"", ""struggles"", and ""joke"" contribute to a strong negative sentiment.",😡
2,"Based on the given examples and the review:\n\n``` \nI tried to like this book and while the plot was really good, the print quality was so not good\n```\n\nThe sentiment would be: 😐\n\nThis is because the reviewer tried to like the book, indicating a positive inclination, but then mentioned a major drawback (print quality), which is a negative aspect. The overall sentiment is neutral, as the reviewer is torn between the two aspects.",😐


## Task 3: Coding Tasks - Python

This prompt tests an LLM's capabilities for generating python code based on various tasks


In [18]:
prompt = f"""
Act as an expert in generating python code

Your task is to generate python code
to explain how to scale data for a ML problem.
Focus on just scaling and nothing else.
Keep into account key operations we should do on the data
to prevent data leakage before scaling.
Keep the code and answer concise.
"""

Trying with Llama 3.1

In [19]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Scaling Data for Machine Learning**
=====================================

To scale data for a machine learning problem, we'll use the `StandardScaler` from scikit-learn. Here's a concise example of how to do it:

```python
# Import necessary libraries
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load your dataset into a pandas DataFrame
data = pd.read_csv('your_data.csv')

# Split data into features (X) and target (y)
X = data.drop('target_column', axis=1)  # replace 'target_column' with your actual target column
y = data['target_column']

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform both the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Now you can use the scaled data for your machine learning model
```

**Key Operations:**

1. **Split data into features (X) and target (y)**: This is crucial to prevent data leakage.
2. **Split data into training and testing sets**: Again, this prevents data leakage.
3. **Fit the scaler to the training data**: This is where the scaler learns the mean and standard deviation of the training data.
4. **Transform both the training and testing data**: This applies the scaling to both datasets.

By following these steps, you'll ensure that your data is properly scaled for your machine learning model.

Trying with GPT-4o mini

In [20]:
response = get_completion_gpt(prompt)
display(Markdown(response))

Certainly! Below is a concise Python code snippet that demonstrates how to scale data for a machine learning problem while preventing data leakage. We'll use `StandardScaler` from `sklearn` as an example of scaling.

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Sample data
data = pd.DataFrame({
    'feature1': [1, 2, 3, 4, 5],
    'feature2': [10, 20, 30, 40, 50],
    'target': [0, 1, 0, 1, 0]
})

# Step 1: Split the data into training and testing sets
X = data[['feature1', 'feature2']]
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize the scaler
scaler = StandardScaler()

# Step 3: Fit the scaler only on the training data
X_train_scaled = scaler.fit_transform(X_train)

# Step 4: Transform the test data using the fitted scaler
X_test_scaled = scaler.transform(X_test)

# X_train_scaled and X_test_scaled are now scaled without data leakage
```

### Key Operations:
1. **Data Splitting**: Always split your data into training and testing sets before scaling to prevent data leakage.
2. **Fitting the Scaler**: Fit the scaler only on the training data.
3. **Transforming the Test Data**: Use the fitted scaler to transform the test data, ensuring that the test set remains unseen during the training phase.

## Task 4: Coding Tasks - SQL

This prompt tests an LLM's capabilities for generating SQL code based on various tasks

In [21]:
prompt = f"""
Act as an expert in generating SQL code.

Understand the following schema of the database tables carefully:
Table departments, columns = [DepartmentId, DepartmentName]
Table employees, columns = [EmployeeId, EmployeeName, DepartmentId]
Table salaries, columns = [EmployeeId, Salary]

Create a MySQL query for the employee with the 2nd highest salary in the 'IT' Department.
Output should have EmployeeId, EmployeeName, DepartmentName, Salary
"""

Trying with Llama 3.1

In [22]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To solve this problem, we can use a subquery to first find the maximum and second maximum salaries in the 'IT' department, and then join the employees and departments tables to get the desired output.

Here's the MySQL query:

```sql
SELECT 
    e.EmployeeId,
    e.EmployeeName,
    d.DepartmentName,
    s.Salary
FROM 
    employees e
JOIN 
    departments d ON e.DepartmentId = d.DepartmentId
JOIN 
    salaries s ON e.EmployeeId = s.EmployeeId
WHERE 
    d.DepartmentName = 'IT'
    AND s.Salary IN (
        SELECT 
            DISTINCT Salary
        FROM 
            salaries
        WHERE 
            EmployeeId IN (
                SELECT 
                    EmployeeId
                FROM 
                    employees
                WHERE 
                    DepartmentName = 'IT'
            )
        ORDER BY 
            Salary DESC
        LIMIT 2
    );
```

However, the query above is not the most efficient way to solve this problem. We can optimize it by using a subquery to get the second highest salary in the 'IT' department, and then join the employees and departments tables to get the desired output.

Here's the optimized query:

```sql
SELECT 
    e.EmployeeId,
    e.EmployeeName,
    d.DepartmentName,
    s.Salary
FROM 
    employees e
JOIN 
    departments d ON e.DepartmentId = d.DepartmentId
JOIN 
    salaries s ON e.EmployeeId = s.EmployeeId
WHERE 
    d.DepartmentName = 'IT'
    AND s.Salary = (
        SELECT 
            DISTINCT Salary
        FROM 
            salaries
        WHERE 
            EmployeeId IN (
                SELECT 
                    EmployeeId
                FROM 
                    employees
                WHERE 
                    DepartmentName = 'IT'
            )
        ORDER BY 
            Salary DESC
        LIMIT 1 OFFSET 1
    );
```

However, MySQL does not support the `LIMIT` clause in a subquery, so we need to use a different approach. We can use a variable to store the maximum salary and then select the employee with the second highest salary.

Here's the query using a variable:

```sql
SET @max_salary = NULL;

SELECT 
    e.EmployeeId,
    e.EmployeeName,
    d.DepartmentName,
    s.Salary
FROM 
    employees e
JOIN 
    departments d ON e.DepartmentId = d.DepartmentId
JOIN 
    salaries s ON e.EmployeeId = s.EmployeeId
WHERE 
    d.DepartmentName = 'IT'
    AND s.Salary = (
        SELECT 
            DISTINCT Salary
        FROM 
            salaries
        WHERE 
            EmployeeId IN (
                SELECT 
                    EmployeeId
                FROM 
                    employees
                WHERE 
                    DepartmentName = 'IT'
            )
        ORDER BY 
            Salary DESC
        LIMIT 1 OFFSET 1
    )
    OR s.Salary = (
        SELECT 
            @max_salary := MAX(Salary)
        FROM 
            salaries
        WHERE 
            EmployeeId IN (
                SELECT 
                    EmployeeId
                FROM 
                    employees
                WHERE 
                    DepartmentName = 'IT'
            )
    );
```

However, the query above is still not the most efficient way to solve this problem. We can optimize it by using a subquery to get the second highest salary in the 'IT' department, and then join the employees and departments tables to get the desired output.

Here's the optimized query using a subquery:

```sql
SELECT 
    e.EmployeeId,
    e.EmployeeName,
    d.DepartmentName,
    s.Salary
FROM 
    employees e
JOIN 
    departments d ON e.DepartmentId = d.DepartmentId
JOIN 
    salaries s ON e.EmployeeId = s.EmployeeId
WHERE 
    d.DepartmentName = 'IT'
    AND s.Salary = (
        SELECT 
            Salary
        FROM 
            (
                SELECT 
                    Salary,
                    DENSE_RANK() OVER (ORDER BY Salary DESC) AS rank
                FROM 
                    salaries
                WHERE 
                    EmployeeId IN (
                        SELECT 
                            EmployeeId
                        FROM 
                            employees
                        WHERE 
                            DepartmentName = 'IT'
                    )
            ) t
        WHERE 
            rank = 2
    );
```

This query uses a subquery to get the second highest salary in the 'IT' department, and then joins the employees and departments tables to get the desired output. This is the most efficient way to solve this problem.

Trying with GPT-4o mini

In [23]:
response = get_completion_gpt(prompt)
display(Markdown(response))

To retrieve the employee with the 2nd highest salary in the 'IT' department, you can use a combination of subqueries and joins. Here’s the SQL query that accomplishes this:

```sql
SELECT e.EmployeeId, e.EmployeeName, d.DepartmentName, s.Salary
FROM employees e
JOIN departments d ON e.DepartmentId = d.DepartmentId
JOIN salaries s ON e.EmployeeId = s.EmployeeId
WHERE d.DepartmentName = 'IT'
ORDER BY s.Salary DESC
LIMIT 1 OFFSET 1;
```

### Explanation:
1. **Joins**: The query joins the `employees`, `departments`, and `salaries` tables to get the necessary information.
2. **WHERE Clause**: It filters the results to include only employees from the 'IT' department.
3. **ORDER BY**: The results are ordered by `Salary` in descending order to get the highest salaries first.
4. **LIMIT and OFFSET**: The `LIMIT 1 OFFSET 1` clause is used to skip the highest salary (offset 1) and return the next one (limit 1), which corresponds to the 2nd highest salary.

This query will return the `EmployeeId`, `EmployeeName`, `DepartmentName`, and `Salary` of the employee with the 2nd highest salary in the 'IT' department.

## Task 5: Information Extraction

This prompt tests an LLM's capabilities for extracting and analyzing key entities from documents

In [24]:
clinical_note = """
60-year-old man in NAD with a h/o CAD, DM2, asthma, pharyngitis, SBP,
and HTN on altace for 8 years awoke from sleep around 1:00 am this morning
with a sore throat and swelling of the tongue.
He came immediately to the ED because he was having difficulty swallowing and
some trouble breathing due to obstruction caused by the swelling.
He did not have any associated SOB, chest pain, itching, or nausea.
He has not noticed any rashes.
He says that he feels like it is swollen down in his esophagus as well.
He does not recall vomiting but says he might have retched a bit.
In the ED he was given 25mg benadryl IV, 125 mg solumedrol IV,
and pepcid 20 mg IV.
Family history of CHF and esophageal cancer (father).
"""

In [25]:
prompt = f"""
Act as an expert in analyzing and understanding clinical doctor notes in healthcare.
Extract all symptoms only from the clinical note below in triple backticks.

Differentiate between symptoms that are present vs. absent.
Give me the probability (high/ medium/ low) of how sure you are about the result.
Add a note on the probabilities and why you think so.

Output as a markdown table with the following columns,
all symptoms should be expanded and no acronyms unless you don't know:

Symptoms | Present/Denies | Probability.


Also expand the acronyms in the note including symptoms and other medical terms.
Do not leave out any acronym related to healthcare.

Output that also as a separate appendix table in Markdown with the following columns,

Acronym | Expanded Term

Clinical Note:
```{clinical_note}```
"""

Trying with Llama 3.1

In [26]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Symptoms Table**

| Symptoms | Present/Denies | Probability |
| --- | --- | --- |
| Sore throat | Present | High |
| Swelling of the tongue | Present | High |
| Difficulty swallowing | Present | High |
| Trouble breathing | Present | High |
| Obstruction caused by swelling | Present | High |
| Shortness of breath (SOB) | Absent | Medium |
| Chest pain | Absent | Medium |
| Itching | Absent | Low |
| Nausea | Absent | Low |
| Rashes | Absent | Low |
| Vomiting | Absent | Low |
| Retching | Present | Medium |
| Swelling in the esophagus | Present | High |

**Note on Probabilities:**

I have assigned high probability to symptoms that are directly mentioned in the clinical note, such as sore throat, swelling of the tongue, difficulty swallowing, and trouble breathing. These symptoms are explicitly stated by the patient and are consistent with the patient's presentation.

I have assigned medium probability to symptoms that are partially mentioned or have some ambiguity, such as shortness of breath (SOB) and retching. The patient does not explicitly deny SOB, but it is not mentioned as a symptom. The patient also mentions retching, but it is not clear if it is a symptom or a consequence of the swelling.

I have assigned low probability to symptoms that are explicitly denied or not mentioned at all, such as itching, nausea, and rashes.

**Appendix Table: Expanded Acronyms**

| Acronym | Expanded Term |
| --- | --- |
| NAD | Normal and Duly (Note: In medical context, it's more common to see "NAD" as Normal and Duly, but it can also be "Normal and Duly" or "Normal and Drunk" depending on context) |
| CAD | Coronary Artery Disease |
| DM2 | Diabetes Mellitus Type 2 |
| SBP | Systolic Blood Pressure |
| HTN | Hypertension |
| SOB | Shortness of Breath |
| ED | Emergency Department |
| IV | Intravenous |
| CHF | Congestive Heart Failure |
| H/o | History of (Note: This is a medical abbreviation, but it's not an acronym. It's more common to see "Hx" or "H/O") |
| DM | Diabetes Mellitus (Note: DM is not an acronym, but it's a common abbreviation for Diabetes Mellitus) |
| SBP | Systolic Blood Pressure |
| HTN | Hypertension |
| CAD | Coronary Artery Disease |
| DM2 | Diabetes Mellitus Type 2 |
| HTN | Hypertension |
| CHF | Congestive Heart Failure |
| ED | Emergency Department |
| IV | Intravenous |
| H/o | History of |
| SOB | Shortness of Breath |
| CAD | Coronary Artery Disease |
| HTN | Hypertension |
| DM2 | Diabetes Mellitus Type 2 |
| SBP | Systolic Blood Pressure |
| H/o | History of |
| CHF | Congestive Heart Failure |
| ED | Emergency Department |
| IV | Intravenous |
| SOB | Shortness of Breath |

Trying with GPT-4o mini

In [27]:
response = get_completion_gpt(prompt)
display(Markdown(response))

### Symptoms Analysis

| Symptoms                                   | Present/Denies | Probability |
|--------------------------------------------|----------------|-------------|
| Sore throat                                | Present        | High        |
| Swelling of the tongue                     | Present        | High        |
| Difficulty swallowing                      | Present        | High        |
| Trouble breathing                          | Present        | High        |
| Shortness of breath                        | Denies         | High        |
| Chest pain                                 | Denies         | High        |
| Itching                                    | Denies         | High        |
| Nausea                                     | Denies         | High        |
| Rashes                                      | Denies         | High        |
| Swelling in the esophagus                  | Present        | Medium      |
| Vomiting                                   | Denies         | Medium      |
| Retching                                   | Present        | Medium      |

### Note on Probabilities
The probabilities are assessed based on the clarity and specificity of the symptoms mentioned in the clinical note. Symptoms that are explicitly stated as present (e.g., sore throat, swelling of the tongue) are given a high probability due to direct mention. Symptoms that are denied (e.g., shortness of breath, chest pain) are also assigned a high probability since they are clearly stated as absent. For symptoms like swelling in the esophagus and retching, the probability is medium because they are somewhat subjective and less clearly defined in the context of the note.

### Appendix: Acronyms

| Acronym | Expanded Term                          |
|---------|----------------------------------------|
| NAD     | No Acute Distress                      |
| h/o     | History of                             |
| CAD     | Coronary Artery Disease                |
| DM2     | Type 2 Diabetes Mellitus               |
| SBP     | Systolic Blood Pressure                 |
| HTN     | Hypertension                           |
| IV      | Intravenous                            |
| CHF     | Congestive Heart Failure               | 
| ED      | Emergency Department                   | 
| mg      | Milligram                              |
| solumedrol | Methylprednisolone                  |
| pepcid  | Famotidine                             |

## Task 6: Closed-Domain Question Answering

Question Answering (QA) is a natural language processing task which involves generating the desired answer for the given question. Question Answering can be open-domain QA or closed-domain QA depending on whether the LLM is provided with the relevant context or not.

In the case of closed-domain QA, a question along with relevant context is given. Here the context is nothing but the relevant text which ideally should have the answer. Just like a RAG workflow.

In [28]:
report = """
Three quarters (77%) of the population saw an increase in their regular outgoings over the past year,
according to findings from our recent consumer survey. In contrast, just over half (54%) of respondents
had an increase in their salary, which suggests that the burden of costs outweighing income remains for
most. In total, across the 2,500 people surveyed, the increase in outgoings was 18%, three times higher
than the 6% increase in income.

Despite this, the findings of our survey suggest we have reached a plateau. Looking at savings,
for example, the share of people who expect to make regular savings this year is just over 70%,
broadly similar to last year. Over half of those saving plan to use some of the funds for residential
property. A third are saving for a deposit, and a further 20% for an investment property or second home.

But for some, their plans are being pushed back. 9% of respondents stated they had planned to purchase
a new home this year but have now changed their mind. While for many the deposit may be an issue,
the other driving factor remains the cost of the mortgage, which has been steadily rising the last
few years. For those that currently own a property, the survey showed that in the last year,
the average mortgage payment has increased from £668.51 to £748.94, or 12%."""


In [29]:
question = """
How much has the average mortage payment increased in the last year?
"""

prompt = f"""
Using the following context information below please answer the following question
to the best of your ability
Context:
{report}
Question:
{question}
Answer:
"""

Trying with Llama 3.1

In [30]:
response = get_completion_llama(prompt)
display(Markdown(response))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


According to the context information, the average mortgage payment has increased from £668.51 to £748.94, which is an increase of £80.43.

Trying with GPT-4o mini

In [31]:
response = get_completion_gpt(prompt)
display(Markdown(response))

The average mortgage payment has increased from £668.51 to £748.94 in the last year, which is an increase of £80.43.

In [32]:
question = """
What percentage of people had an increase in salary last year? Show the answer just as a number.
"""

prompt = f"""
Using the following context information below please answer the following question
to the best of your ability
Context:
{report}
Question:
{question}
Answer:
"""

Trying with Llama 3.1

In [33]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


54

Trying with GPT-4o mini

In [34]:
response = get_completion_gpt(prompt)
display(Markdown(response))

54

## Task 7: Open-Domain Question Answering

Question Answering (QA) is a natural language processing task which involves generating the desired answer for the given question.

In the case of open-domain QA, only the question is asked without providing any context or information. Here, the LLM answers the question using the knowledge gained from large volumes of text data during its training. This is basically Zero-Shot QA

In [ ]:
prompt = f"""
Please answer the following question to the best of your ability
Question:
What is LangChain?

Answer:
"""

Trying with Llama 3.1

In [ ]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LangChain is an open-source Python library that enables developers to build conversational AI applications using a modular, chain-based architecture. It provides a set of building blocks, called "chains," that can be combined to create complex conversational flows, allowing for more natural and context-aware interactions.

LangChain's architecture is designed to facilitate the creation of conversational AI systems that can understand and respond to user input in a more human-like way. It achieves this by breaking down the conversation into smaller, manageable components, which can be chained together to create a cohesive and context-aware conversation flow.

Some key features of LangChain include:

1. **Modular architecture**: LangChain's chain-based architecture allows developers to build conversational AI systems by combining smaller, reusable components.
2. **Context-aware conversations**: LangChain enables developers to create conversational flows that are aware of the context and history of the conversation.
3. **Natural language understanding**: LangChain provides a range of natural language processing (NLP) tools and techniques to enable developers to build conversational AI systems that can understand user input.
4. **Integration with other libraries**: LangChain is designed to work seamlessly with other popular AI and NLP libraries, such as Hugging Face Transformers and spaCy.

Overall, LangChain is a powerful tool for building conversational AI applications that can understand and respond to user input in a more natural and context-aware way.

Trying with GPT-4o mini

In [ ]:
response = response = get_completion_gpt(prompt)
display(Markdown(response))

LangChain is a framework designed to facilitate the development of applications that utilize large language models (LLMs). It provides a set of tools and components that help developers build, manage, and deploy applications that can leverage the capabilities of LLMs for various tasks, such as natural language understanding, text generation, and conversational agents.

Key features of LangChain include:

1. **Modularity**: LangChain allows developers to create modular components that can be easily integrated and reused across different applications.

2. **Chain Management**: It supports the creation of "chains," which are sequences of operations that can be executed in a specific order, enabling complex workflows.

3. **Integration with External Data Sources**: LangChain can connect with various data sources, APIs, and databases, allowing applications to pull in relevant information to enhance the capabilities of the language model.

4. **Prompt Management**: The framework provides tools for managing and optimizing prompts, which are crucial for guiding the behavior of LLMs.

5. **Support for Multiple LLMs**: LangChain is designed to work with various language models, making it flexible for developers to choose the best model for their specific use case.

Overall, LangChain aims to simplify the process of building applications that harness the power of language models, making it easier for developers to create innovative solutions in areas like chatbots, content generation, and more.

In [35]:
prompt = f"""
Please answer the following question to the best of your ability
Question:
13.11 or 13.8 which is larger and why?

Answer:
"""

Trying with Llama 3.1

In [36]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The number 13.8 is larger than 13.11.

This is because 13.8 has a higher decimal value than 13.11. In a decimal number, the digits to the right of the decimal point represent fractions of a unit. The more digits to the right of the decimal point, the more precise the number is. 

In this case, 13.8 has a smaller decimal value (0.8) compared to 13.11 (0.11). However, since 0.11 is larger than 0.8, 13.11 is larger than 13.8.

Trying with GPT-4o mini

In [37]:
response = response = get_completion_gpt(prompt)
display(Markdown(response))

13.11 is larger than 13.8. This is because when comparing decimal numbers, you start from the leftmost digit. In this case, both numbers have the same whole number part (13). However, when you compare the tenths place, 1 (in 13.11) is greater than 0 (in 13.8). Therefore, 13.11 is larger than 13.8.

## Task 8: Document Summarization

Document summarization is a natural language processing task which involves creating a concise summary of the given text, while still capturing all the important information.

In [ ]:
doc = """
Coronaviruses are a large family of viruses which may cause illness in animals or humans.
In humans, several coronaviruses are known to cause respiratory infections ranging from the
common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS).
The most recently discovered coronavirus causes coronavirus disease COVID-19.
COVID-19 is the infectious disease caused by the most recently discovered coronavirus.
This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019.
COVID-19 is now a pandemic affecting many countries globally.
The most common symptoms of COVID-19 are fever, dry cough, and tiredness.
Other symptoms that are less common and may affect some patients include aches
and pains, nasal congestion, headache, conjunctivitis, sore throat, diarrhea,
loss of taste or smell or a rash on skin or discoloration of fingers or toes.
These symptoms are usually mild and begin gradually.
Some people become infected but only have very mild symptoms.
Most people (about 80%) recover from the disease without needing hospital treatment.
Around 1 out of every 5 people who gets COVID-19 becomes seriously ill and develops difficulty breathing.
Older people, and those with underlying medical problems like high blood pressure, heart and lung problems,
diabetes, or cancer, are at higher risk of developing serious illness.
However, anyone can catch COVID-19 and become seriously ill.
People of all ages who experience fever and/or  cough associated with difficulty breathing/shortness of breath,
chest pain/pressure, or loss of speech or movement should seek medical attention immediately.
If possible, it is recommended to call the health care provider or facility first,
so the patient can be directed to the right clinic.
People can catch COVID-19 from others who have the virus.
The disease spreads primarily from person to person through small droplets from the nose or mouth,
which are expelled when a person with COVID-19 coughs, sneezes, or speaks.
These droplets are relatively heavy, do not travel far and quickly sink to the ground.
People can catch COVID-19 if they breathe in these droplets from a person infected with the virus.
This is why it is important to stay at least 1 meter) away from others.
These droplets can land on objects and surfaces around the person such as tables, doorknobs and handrails.
People can become infected by touching these objects or surfaces, then touching their eyes, nose or mouth.
This is why it is important to wash your hands regularly with soap and water or clean with alcohol-based hand rub.
Practicing hand and respiratory hygiene is important at ALL times and is the best way to protect others and yourself.
When possible maintain at least a 1 meter distance between yourself and others.
This is especially important if you are standing by someone who is coughing or sneezing.
Since some infected persons may not yet be exhibiting symptoms or their symptoms may be mild,
maintaining a physical distance with everyone is a good idea if you are in an area where COVID-19 is circulating."""

In [ ]:
prompt = f"""
You are an expert in generating accurate document summaries.
Generate a summary of the given document.

Document:
{doc}


Constraints: Please start the summary with the delimiter 'Summary'
and limit the summary to 5 lines

Summary:
"""

Trying with Llama 3.1

In [ ]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Summary:
Coronaviruses are a large family of viruses that cause respiratory infections in humans, ranging from the common cold to severe diseases like MERS and SARS. COVID-19 is a new infectious disease caused by a recently discovered coronavirus, first identified in Wuhan, China in December 2019. Common symptoms of COVID-19 include fever, dry cough, and tiredness, with some people experiencing mild symptoms. Older people and those with underlying medical conditions are at higher risk of developing serious illness. Practicing hand and respiratory hygiene, maintaining a 1-meter distance from others, and washing hands regularly can help prevent the spread of COVID-19.

Trying with GPT-4o mini

In [ ]:
response = response = get_completion_gpt(prompt)
display(Markdown(response))

Summary: Coronaviruses are a large family of viruses that can cause respiratory infections in humans, including COVID-19, which emerged in December 2019. Common symptoms include fever, dry cough, and tiredness, with serious cases leading to difficulty breathing, particularly in older adults and those with underlying health issues. COVID-19 spreads primarily through respiratory droplets from infected individuals. Preventative measures include maintaining physical distance, practicing hand hygiene, and seeking medical attention for severe symptoms.

## Task 9: Transformation

You can use LLMs to take an existing document and transform it into other formats of content and even generate training data for fine-tuning or training models

In [ ]:
fact_sheet_mobile = """
PRODUCT NAME
Samsung Galaxy Z Fold4 5G Black
​
PRODUCT OVERVIEW
Stands out. Stands up. Unfolds.
The Galaxy Z Fold4 does a lot in one hand with its 15.73 cm(6.2-inch) Cover Screen.
Unfolded, the 19.21 cm(7.6-inch) Main Screen lets you really get into the zone.
Pushed-back bezels and the Under Display Camera means there's more screen
and no black dot getting between you and the breathtaking Infinity Flex Display.
Do more than more with Multi View. Whether toggling between texts or catching up
on emails, take full advantage of the expansive Main Screen with Multi View.
PC-like power thanks to Qualcomm Snapdragon 8+ Gen 1 processor in your pocket,
transforms apps optimized with One UI to give you menus and more in a glance
New Taskbar for PC-like multitasking. Wipe out tasks in fewer taps. Add
apps to the Taskbar for quick navigation and bouncing between windows when
you're in the groove.4 And with App Pair, one tap launches up to three apps,
all sharing one super-productive screen
Our toughest Samsung Galaxy foldables ever. From the inside out,
Galaxy Z Fold4 is made with materials that are not only stunning,
but stand up to life's bumps and fumbles. The front and rear panels,
made with exclusive Corning Gorilla Glass Victus+, are ready to resist
sneaky scrapes and scratches. With our toughest aluminum frame made with
Armor Aluminum, this is one durable smartphone.
World’s first water resistant foldable smartphones. Be adventurous, rain
or shine. You don't have to sweat the forecast when you've got one of the
world's first water-resistant foldable smartphones.
​
PRODUCT SPECS
OS - Android 12.0
RAM - 12 GB
Product Dimensions - 15.5 x 13 x 0.6 cm; 263 Grams
Batteries - 2 Lithium Ion batteries required. (included)
Item model number - SM-F936BZKDINU_5
Wireless communication technologies - Cellular
Connectivity technologies - Bluetooth, Wi-Fi, USB, NFC
GPS - True
Special features - Fast Charging Support, Dual SIM, Wireless Charging, Built-In GPS, Water Resistant
Other display features - Wireless
Device interface - primary - Touchscreen
Resolution - 2176x1812
Other camera features - Rear, Front
Form factor - Foldable Screen
Colour - Phantom Black
Battery Power Rating - 4400
Whats in the box - SIM Tray Ejector, USB Cable
Manufacturer - Samsung India pvt Ltd
Country of Origin - China
Item Weight - 263 g
"""

In [ ]:
prompt =f"""Turn the following product description
into a list of frequently asked questions (FAQ).
Show both the question and its corresponding answer
Generate at the max 5 but diverse and useful FAQs

Product description:
```{fact_sheet_mobile}```
"""

Trying with Llama 3.1

In [ ]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here are 5 diverse and useful FAQs based on the product description:

**Q1: What is the screen size of the Galaxy Z Fold4 5G?**
A1: The Galaxy Z Fold4 5G has a 15.73 cm (6.2-inch) Cover Screen and a 19.21 cm (7.6-inch) Main Screen when unfolded.

**Q2: How durable is the Galaxy Z Fold4 5G?**
A2: The Galaxy Z Fold4 5G is made with the toughest materials, including Corning Gorilla Glass Victus+ and Armor Aluminum, which provide excellent resistance to scratches and scrapes.

**Q3: Can I use the Galaxy Z Fold4 5G in the rain?**
A3: Yes, the Galaxy Z Fold4 5G is one of the world's first water-resistant foldable smartphones, so you can use it in the rain without worrying about damage.

**Q4: What kind of multitasking capabilities does the Galaxy Z Fold4 5G offer?**
A4: The Galaxy Z Fold4 5G offers PC-like multitasking capabilities with features like Multi View, New Taskbar, and App Pair, which allow you to easily switch between apps and windows.

**Q5: What kind of processor powers the Galaxy Z Fold4 5G?**
A5: The Galaxy Z Fold4 5G is powered by a Qualcomm Snapdragon 8+ Gen 1 processor, which provides fast performance and efficient battery life.

Trying with GPT-4o mini

In [ ]:
response = response = get_completion_gpt(prompt)
display(Markdown(response))

### Frequently Asked Questions (FAQ)

**Q1: What are the main features of the Samsung Galaxy Z Fold4 5G?**  
**A1:** The Samsung Galaxy Z Fold4 features a 6.2-inch Cover Screen and a 7.6-inch Main Screen with an Infinity Flex Display. It includes a Qualcomm Snapdragon 8+ Gen 1 processor, 12 GB of RAM, Multi View for multitasking, and a new Taskbar for easy navigation. It is also water-resistant and made with durable materials like Corning Gorilla Glass Victus+ and Armor Aluminum.

---

**Q2: Is the Samsung Galaxy Z Fold4 water-resistant?**  
**A2:** Yes, the Samsung Galaxy Z Fold4 is one of the world’s first water-resistant foldable smartphones, allowing you to use it in various weather conditions without worry.

---

**Q3: What is the battery capacity of the Samsung Galaxy Z Fold4?**  
**A3:** The Samsung Galaxy Z Fold4 is equipped with a battery power rating of 4400 mAh, and it supports fast charging, wireless charging, and dual SIM functionality.

---

**Q4: What operating system does the Samsung Galaxy Z Fold4 run on?**  
**A4:** The Samsung Galaxy Z Fold4 runs on Android 12.0, optimized with One UI for enhanced performance and multitasking capabilities.

---

**Q5: What accessories are included in the box with the Samsung Galaxy Z Fold4?**  
**A5:** The box includes a SIM Tray Ejector and a USB Cable.

## Task 10: Translation

You can use LLMs to take an existing document and translate it from a source to target language. You can also translate to multiple languages at the same time

In [ ]:
prompt = """You are an expert translator.
Translate the given text from English to German and Spanish.
Show the output as key value pairs in JSON.
Output should have all 3 languages.

Text: 'Hello, how are you today?'
Translation:
"""

Trying with Llama 3.1

In [ ]:
response = get_completion_llama(prompt)
display(Markdown(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here is the translation of the given text in English to German and Spanish as key-value pairs in JSON format:

```json
{
  "English": "Hello, how are you today?",
  "German": "Hallo, wie geht es Ihnen heute?",
  "Spanish": "Hola, ¿cómo estás hoy?"
}
```

Note: The translations are provided in a formal tone. If you need informal translations or any variations, please let me know!

Trying with GPT-4o mini

In [ ]:
response = response = get_completion_gpt(prompt)
display(Markdown(response))

```json
{
  "English": "Hello, how are you today?",
  "German": "Hallo, wie geht es dir heute?",
  "Spanish": "Hola, ¿cómo estás hoy?"
}
```